In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Sequential
import datetime



In [2]:
data_train = pd.read_csv("train_dataset_train.csv")

In [3]:
data_train.head(3)

,id,Easting,Northing,Height,Reflectance,Class
0,2321251,431696.5375,6.032319e+06,69.2226,-11.14,0
1,3515173,431710.3835,6.032291e+06,68.9711,-15.16,3
2,2320295,431696.8099,6.032322e+06,69.2453,-13.59,0


In [4]:
data_y = data_train["Class"]
encod_y = LabelBinarizer()
Y_train = encod_y.fit_transform(data_y)

In [5]:
def get_train_dataset_x(data_x):

  data_X = data_x.copy()
  norm = MinMaxScaler()
  pca = PCA(4)
  res = norm.fit_transform(data_X)
  res = pca.fit_transform(res)
  X_train = res

  return X_train, norm, pca

In [6]:
def get_test_dataset_x(data_x, norm, pca):

  data_X = data_x.copy()
  res = norm.transform(data_X)
  res = pca.transform(res)
  X_test = res

  return X_test

In [7]:
data_x = data_train[data_train.columns[1:-1]]
X_train, norm, pca = get_train_dataset_x(data_x)
X_train

array([[-1.01947707e-01,  7.77531240e-03,  1.64686656e-02,
        -1.29582064e-03],
       [ 9.89887398e-02,  1.19426195e-01,  2.63367159e-02,
        -2.15209267e-02],
       [-1.13230124e-01,  4.11570049e-02,  6.64589407e-02,
        -7.40197084e-05],
       ...,
       [ 2.01215748e-01, -4.95655093e-02,  4.00433451e-02,
         5.44865668e-03],
       [-6.97160688e-02, -2.61389949e-02,  1.08523052e-02,
         3.99274269e-03],
       [-1.29094245e-01, -2.11752073e-01,  4.45808071e-02,
         3.88453101e-03]])

In [8]:
X_train, norm, pca = get_train_dataset_x(data_x)

#Создание модели обучения

In [10]:
import tensorflow as tf
mods = []
hists = []
num = 0
k = 600
act = "relu"
end = "softmax"
metric = tf.keras.metrics.Recall()
model = Sequential()
model.add(Dense(k, activation=act, input_shape = (X_train.shape[1],)))
model.add(Dense(k, activation=act))
model.add(Dense(Y_train.shape[1], activation=end))
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=[metric])

In [12]:
history = model.fit(X_train, Y_train, epochs=10, validation_split = 0.3, batch_size=1000, verbose = 1)

Epoch 1/10
2956/2956 [==============================] - 33s 11ms/step - loss: 0.0692 - recall_1: 0.9717 - val_loss: 0.0719 - val_recall_1: 0.9695

In [23]:
test = pd.read_csv("test_dataset_test.csv")
X_test = get_test_dataset_x(test[test.columns[1:]], norm, pca)
Y_test = model.predict(X_test, batch_size = 1000, verbose=1)

1810/1810 [==============================] - 3s 2ms/step

In [31]:
classif = encod_y.inverse_transform(Y_test)
id = test["id"].to_numpy()
res = list(zip(id,classif))
out = pd.DataFrame(res, columns=["id", "Class"])
out.to_csv(f"result.csv", index=False)

In [32]:
model.save(f"model.h5")